In [ ]:
File dataFile = new File("/Users/djd/dev/jax/ckb-utilities/ckb-utilities/console-scripts/CKBU-575/gene_category_sunburst_data.json")
dataFile.text
def slurper = new groovy.json.JsonSlurper()
json =slurper.parseText(dataFile.text)
beakerx.ids = json.ids
beakerx.labels = json.labels
beakerx.parents = json.parents
beakerx.eeCounts = json.values
beakerx.colors = json.colors
beakerx.labelToIndexesMap = json.labelToIndexesMap
beakerx.labelsWithMultipleIndexes = json.labelsWithMultipleIndexes

In [ ]:
%%javascript
require.config({paths:{'plotly':'//cdn.plot.ly/plotly-latest.min'}});
require(['plotly'], function(Plotly) {
    window.Plotly = Plotly;
});

In [ ]:
%%javascript

beakerx.displayHTML(this, '<div id="plotlySunburst"/>');

var data = [{
  type: "sunburst",
  name: "",
  ids: beakerx.ids,
  labels: beakerx.labels,
  parents: beakerx.parents,
  values: beakerx.eeCounts.map(x => ((x > 0) ? x : 1)),
  labelToIndexesMap: beakerx.labelToIndexesMap,
  labelsWithMultipleIndexes: beakerx.labelsWithMultipleIndexes,
  customdata: beakerx.eeCounts,
  branchvalues: "remainder",
  outsidetextfont: {size: 20, color: "#377eb8"},
  leaf: {opacity: 1},
  marker: {
      line: {width: 2},
      colors: beakerx.colors,
  },
  hovertemplate:'<b>Label</b>: %{label}<br>'  +           
                 '<b>EE Count</b>: %{customdata}<br>' +
                 '<b>Path</b>: %{id}<br>',
//                  '<extra>%{currentPath}</extra>',
//   texttemplate: '%{label} (%{customdata})',
}];

var layout = {
  margin: {l: 0, r: 0, b: 0, t: 0},
  width: 1200,
  height: 1200,
  updatemenus: [
      {
          buttons: 
          [
            {args: ['type', 'sunburst'],label: 'sunburst',method: 'restyle'},
            {args: ['type', 'treemap'],label: 'treemap',method: 'restyle'},
            {args: ['level', ''],label: 'Show Root Level',method: 'restyle'},
          ],          
          type: 'buttons',          
      }
  ],
};


Plotly.newPlot('plotlySunburst', data, layout);

var myPlot = document.getElementById('plotlySunburst');

myPlot.on('plotly_hover', function(data){   
    const point = data.points[0];
    const plotData = point.data;
    const label = point.label;
    //console.log(plotData);
    console.log("label: " + label);    
    if(plotData.labelsWithMultipleIndexes.includes(label)){
        const indexesForIds = plotData.labelToIndexesMap[label];
        console.log("indexesForIds: " + indexesForIds);
        indexesForIds.forEach(index => plotData.marker.colors[index] = 'red');
        Plotly.restyle(myPlot, plotData.marker);
    }        
});
myPlot.on('plotly_unhover', function(data){   
    const point = data.points[0];
    const plotData = point.data;
    const label = point.label;
    if(plotData.labelsWithMultipleIndexes.includes(label)){
        const indexesForIds = plotData.labelToIndexesMap[label];
        indexesForIds.forEach(index => plotData.marker.colors[index] = '');
        Plotly.restyle(myPlot, plotData.marker);    
    }    
});

